In [ ]:
# import re
# pattern=u'[\u0980-\u09FF]+'
# bangla_words=re.findall(pattern, text)

# #=====
# pattern=u'[\u0600-\u06FF]+'


In [3]:
# Import the modules
import re 
import json
import glob
import os 
from tqdm import tqdm
from langdetect import detect
src_dir = "/media/rezwan/Study/Bengali_AI/hadith/hadith/"
dest_dir = "/media/rezwan/Study/Bengali_AI/hadith/process_hadith/"
hadith_books = ['bukhari', 'muslim']

for hadith in hadith_books:
    read_json_dir = os.path.join(src_dir, hadith)
    write_json_dir = os.path.join(dest_dir, hadith)
    for file in tqdm(sorted(glob.glob(os.path.join(read_json_dir, "*")))):
        # print(file)
        # Opening JSON file
        f = open(file)
        # Convert JSON string to dictionary
        data = json.load(f)
        data_demo = data.copy()
        '''
        TO-DO:
            "bangla": {
                        "translation": translation,
                        "reference": ref
                    }
        '''
        # Bangla text 
        bangla_text=data['data']['bangla']
        # print(text)
        bangla_dict = {}
        translator = None
        ref = None 
        if bangla_text != None:
            # "(?<!\.)\n\n"
            newline_pattern=u'(?<!\.)\n\n' #\n\n
            translator_ref=re.split(newline_pattern, bangla_text)
            if len(translator_ref) > 1:
                ref = translator_ref[-1]
                translator = "".join(translator_ref[0:-1])

        bangla_dict['translator'] = translator
        bangla_dict['reference'] = ref
        # insert 
        data_demo['data']['bangla'] = bangla_dict
        # data_demo

        '''
        TO-DO:
            "explanation":{   
                            "para-1":{"text":text,"lang":bn},
                            "para-2":{"text":text,"lang":ar}
                        }
        '''
        # explanation
        explanation_dict = {}
        explanation_text = data['data']['explanation']
        # print(explanation_text)
        if explanation_text != None:
            newline_pattern=u'(?<!\.)\n\n' #\n\n
            explanation_split_text=re.split(newline_pattern, explanation_text)
            for idx, exp_txt in enumerate(explanation_split_text):
                para_no = "para-"+str(idx+1)
                para_dict = {}
                text = exp_txt
                lang = detect(exp_txt)
                para_dict['text'] = text
                para_dict['lang'] = lang
                explanation_dict[para_no] = para_dict

            data_demo['data']['explanation'] = explanation_dict

        ## Save file
        file_name = file.split("/")[-1]
        # print(os.path.join(write_json_dir, file_name))
        with open(os.path.join(write_json_dir, file_name), 'w', encoding='utf8') as json_file:
            json.dump(data_demo, json_file, ensure_ascii=False)

100%|██████████| 7500/7500 [10:06<00:00, 12.37it/s]  
